# TD noté, 16 décembre 2016

In [ ]:
from jyquickhelper import add_notebook_menu
add_notebook_menu()

## Exercice 1

On suppose qu'on dispose d'un ensemble d'observations $(X_i, Y_i)$ avec $X_i, Y_i \in \mathbb{R}$. La régression linéaire consiste une relation linéaire $Y_i = a X_i + b + \epsilon_i$ qui minimise la variance du bruit. On pose :

$$E(a, b) = \sum_i (Y_i - (a X_i + b))^2$$

On cherche $a, b$ tels que :

$$a^*, b^* = \arg \min E(a, b) = \arg \min \sum_i (Y_i - (a X_i + b))^2$$

La fonction est dérivable et on trouve :

$$\frac{\partial E(a,b)}{\partial a} = - 2 \sum_i X_i ( Y_i - (a X_i + b)) \text{ et } \frac{\partial E(a,b)}{\partial b} = - 2 \sum_i ( Y_i - (a X_i + b))$$

Il suffit alors d'annuler les dérivées. On résoud un système d'équations linéaires. On note :

$$\begin{array}{l} \mathbb{E} X = \frac{1}{n}\sum_{i=1}^n X_i \text{ et } \mathbb{E} Y = \frac{1}{n}\sum_{i=1}^n Y_i \\ \mathbb{E}(X^2) = \frac{1}{n}\sum_{i=1}^n X_i^2 \text{ et } \mathbb{E}(XY) = \frac{1}{n}\sum_{i=1}^n X_i Y_i \end{array}$$

Finalement :

$$\begin{array}{l} a^* = \frac{ \mathbb{E}(XY) - \mathbb{E} X \mathbb{E} Y}{\mathbb{E}(X^2) - (\mathbb{E} X)^2} \text{ et } b^* = \mathbb{E} Y - a^* \mathbb{E} X \end{array}$$

### Q1

On génère un nuage de points avec le code suivant :

In [ ]:
import random
def generate_xy(n=100, a=0.5, b=1):
    res = []
    for i in range(0, n):
        x = random.uniform(0, 10)
        res.append((x, x*a + b + random.gauss(0,1)))
    return res

generate_xy(10)

[(0.36916056549562315, 0.9651723331953035),
 (1.3894034176544534, 2.02043819555066),
 (3.4772523254718903, 4.120150217123784),
 (0.918653355682163, 1.394817507218838),
 (9.892187516839433, 3.751344761975619),
 (6.504678926796671, 3.9126056453281857),
 (6.896316004999285, 3.6639729994967687),
 (6.367219487224859, 3.3913298344905876),
 (6.9520777602670325, 5.074143123092575),
 (8.035400726656368, 6.66020581728626)]

### Q2

Ecrire une fonction qui calcule $\mathbb{E} X, \mathbb{E} Y, \mathbb{E}(XY), \mathbb{E}(X^2)$. Plusieurs étudiants m'ont demandé ce qu'était ``obs``. C'est simplement le résultat de la fonction précédente.

In [ ]:
def calcule_exyxyx2(obs):
    sx = 0
    sy = 0
    sxy = 0
    sx2 = 0
    for x, y in obs:
        sx += x
        sy += y
        sxy += x * y
        sx2 += x * x
    n = len(obs)
    return sx/n, sy/n, sxy/n, sx2/n

obs = generate_xy(10)
calcule_exyxyx2(obs)

(4.307497908762707, 3.0353150363290897, 18.58846259018851, 28.129414248382027)

### Q3

 Calculer les grandeurs $a^*$, $b^*$. A priori, on doit retrouver quelque chose d'assez proche des valeurs choisies pour la première question : $a=0.5$, $b=1$.

In [ ]:
def calcule_ab(obs):
    sx, sy, sxy, sx2 = calcule_exyxyx2(obs)
    a = (sxy - sx * sx)  / (sx2 - sx**2)
    b = sy - a * sx
    return a, b

calcule_ab(obs)

(0.003543059580347518, 3.020053314596121)

### Q4

Compléter le programme.

In [ ]:
import random
def generate_caty(n=100, a=0.5, b=1, cats=["rouge", "vert", "bleu"]):
    res = []
    for i in range(0, n):
        x = random.randint(0,2)
        cat = cats[x]
        res.append((cat, 10*x**2*a + b + random.gauss(0,1)))
    return res

generate_caty(10)

[('bleu', 19.99447726888312),
 ('rouge', 2.0344721841590054),
 ('vert', 7.564105414469568),
 ('bleu', 20.491310738223564),
 ('bleu', 22.150799183073342),
 ('bleu', 21.34971890419765),
 ('rouge', 2.7770861108018066),
 ('rouge', 0.8662160890333384),
 ('rouge', -0.1372016892094956),
 ('rouge', 1.8233850064738055)]

### Q5

On voudrait quand même faire une régression de la variable $Y$ sur la variable catégorielle. On construit une fonction qui assigne un numéro quelconque mais distinct à chaque catégorie. La fonction retourne un dictionnaire avec les catégories comme clé et les numéros comme valeurs.

In [ ]:
def numero_cat(obs):
    mapping = {}
    for color, y in obs:
        if color not in mapping:
            mapping[color] = len(mapping)
    return mapping

obs = generate_caty(100)
numero_cat(obs)

{'bleu': 1, 'rouge': 2, 'vert': 0}

### Q6 

On construit la matrice $M_{ic}$ tel que : $M_{ic}$ vaut 1 si $c$ est le numéro de la catégorie $X_i$, 0 sinon.

In [ ]:
import numpy
def construit_M(obs):
    mapping = numero_cat(obs)
    M = numpy.zeros((len(obs), 3))
    for i, (color, y) in enumerate(obs):
        cat = mapping[color]
        M[i, cat] = 1.0
    return M
    
M = construit_M(obs)
M[:5]

array([[ 1.,  0.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  0.,  1.],
       [ 0.,  1.,  0.],
       [ 1.,  0.,  0.]])

### Q7 

Il est conseillé de convertir la matrice $M$ et les $Y$ au format *numpy*. On ajoute un vecteur constant à la matrice $M$. La requête *numpy add column* sur un moteur de recherche vous directement à ce résultat : [How to add an extra column to an numpy array](http://stackoverflow.com/questions/8486294/how-to-add-an-extra-column-to-an-numpy-array).

In [ ]:
def convert_numpy(obs):
    M = construit_M(obs)
    Mc = numpy.hstack([M, numpy.ones((M.shape[0], 1))])
    Y = numpy.array([y for c, y in obs])
    return M, Mc, Y.reshape((M.shape[0], 1))

M, Mc, Y = convert_numpy(obs)
Mc[:5], Y[:5]

(array([[ 1.,  0.,  0.,  1.],
        [ 0.,  1.,  0.,  1.],
        [ 0.,  0.,  1.,  1.],
        [ 0.,  1.,  0.,  1.],
        [ 1.,  0.,  0.,  1.]]), array([[  7.11207585],
        [ 20.29221984],
        [  1.19260385],
        [ 22.04203216],
        [  4.61961025]]))

### Q8 

On résoud la régression multidimensionnelle en appliquant la formule $C^* = (M'M)^{-1}M'Y$. La question 7 ne servait pas à grand chose excepté faire découvrir la fonction [hstack](https://docs.scipy.org/doc/numpy/reference/generated/numpy.hstack.html) car le rang de la matrice ``Mc`` est 3 < 4.

In [ ]:
alpha = numpy.linalg.inv(M.T @ M) @ M.T @ Y
alpha

array([[  5.91759879],
       [ 21.09304778],
       [  1.00360769]])

### Q9 

La régression détermine les coefficients $\alpha$ dans la régression $Y_i = \alpha_{rouge} \mathbb{1}_{X_i = rouge} + \alpha_{vert}  \mathbb{1}_{X_i = vert} + \alpha_{bleu} \mathbb{1}_{X_i = bleu} + \epsilon_i$.

Construire le vecteur $\hat{Y_i} = \alpha_{rouge} \mathbb{1}_{X_i = rouge} + \alpha_{vert} \mathbb{1}_{X_i = vert} + \alpha_{bleu} \mathbb{1}_{X_i = bleu}$.

In [ ]:
Yp = numpy.zeros((M.shape[0], 1))
for i in range(3):
    Yp[ M[:,i] == 1, 0] = alpha[i, 0]
Yp[:5]

array([[  5.91759879],
       [ 21.09304778],
       [  1.00360769],
       [ 21.09304778],
       [  5.91759879]])

### Q10

Utiliser le résultat de la question 3 pour calculer les coefficients de la régression $Y_i = a^* \hat{Y_i} + b^*$.

In [ ]:
obs = [(x, y) for x, y in zip(Yp, Y)]
calcule_ab( obs )

(array([ 1.]), array([ -1.06581410e-14]))

On aboutit au résultat $Y = \hat{Y} + \epsilon$. On a associé une valeur à chaque catégorie de telle sorte que la régression de $Y$ sur cette valeur soit cette valeur. Autrement dit, c'est la meilleur approximation de $Y$ sur chaque catégorie. A quoi cela correspond-il ? C'est le second énoncé qui répond à cette question.

## Exercice 2

### Q1 - Q2 - Q3

Ce sont les mêmes réponses.

### Q4

Un moyen très simple de simuler une [loi multinomiale](https://fr.wikipedia.org/wiki/Loi_multinomiale) est de partir d'une loi uniforme et discrète à valeur dans entre 1 et 10. On tire un nombre, s'il est inférieur ou égal à 5, ce sera la catégorie 0, 1 si c'est inférieur à 8, 2 sinon.

In [ ]:
def generate_caty(n=100, a=0.5, b=1, cats=["rouge", "vert", "bleu"]):
    res = []
    for i in range(0, n):
        # on veut 50% de rouge, 30% de vert, 20% de bleu
        x = random.randint(1, 10)
        if x <= 5: x = 0
        elif x <= 8: x = 1
        else : x = 2
        cat = cats[x]
        res.append((cat, 10*x**2*a + b + random.gauss(0,1)))
    return res

obs = generate_caty(10)
obs

[('bleu', 22.160690197215427),
 ('bleu', 18.6721411554557),
 ('vert', 5.847701207200886),
 ('rouge', -0.21059849120232932),
 ('rouge', 1.5118983060239415),
 ('bleu', 21.010287082691725),
 ('rouge', 0.6678988983315497),
 ('rouge', 1.850800799524975),
 ('vert', 6.761906815318741),
 ('bleu', 20.319905475752662)]

### Q5 

On voudrait quand même faire une régression de la variable $Y$ sur la variable catégorielle. On commence par les compter. Construire une fonction qui compte le nombre de fois qu'une catégorie est présente dans les données (un histogramme).

In [ ]:
def histogram_cat(obs):
    h = dict()
    for color, y in obs:
        h[color] = h.get(color, 0) + 1
    return h

histogram_cat(obs)

{'bleu': 4, 'rouge': 4, 'vert': 2}

### Q6

Construire une fonction qui calcule la moyenne des $Y_i$ pour chaque catégorie : $\mathbb{E}(Y | rouge)$, $\mathbb{E}(Y | vert)$, $\mathbb{E}(Y | bleu)$. La fonction retourne un dictionnaire ``{couleur:moyenne}``. 

In [ ]:
def moyenne_cat(obs):
    h = dict()
    sy = dict()
    for color, y in obs:
        h[color] = h.get(color, 0) + 1
        sy[color] = sy.get(color, 0) + y
    for k, v in h.items():
        sy[k] /= v
    return sy

moyenne_cat(obs)

{'bleu': 20.54075597777888,
 'rouge': 0.9549998781695341,
 'vert': 6.304804011259813}

L'énoncé induisait quelque peu en erreur car la fonction suggérée ne permet de calculer ces moyennes. Il suffit de changer.

### Q7

Construire le vecteur $Z_i = \mathbb{E}(Y | rouge)\mathbb{1}_{X_i = rouge} + \mathbb{E}(Y | vert) \mathbb{1}_{X_i = vert} + \mathbb{E}(Y | bleu) \mathbb{1}_{X_i = bleu}$.

In [ ]:
moys = moyenne_cat(obs)
Z = [moys[c] for c, y in obs]
Z[:5]

[20.54075597777888,
 20.54075597777888,
 6.304804011259813,
 0.9549998781695341,
 0.9549998781695341]

### Q8

Utiliser le résultat de la question 3 pour calculer les coefficients de la régression $Y_i = a^* Z_i + b^*$.

In [ ]:
obs2 = [(z, y) for (c, y), z in zip(obs, Z)]
calcule_ab( obs2 )

(1.0000000000000004, -3.552713678800501e-15)

On aboutit au résultat $Y = \hat{Y} + \epsilon$. On a associé une valeur à chaque catégorie de telle sorte que la régression de $Y$ sur cette valeur soit cette valeur.

### Q9 

Calculer la matrice de variance / covariance pour les variables $(Y_i)$, $(Z_i)$, $(Y_i - Z_i)$, $\mathbb{1}_{X_i = rouge}$, $\mathbb{1}_{X_i = vert}$, $\mathbb{1}_{X_i = bleu}$. 

In [ ]:
bigM = numpy.empty((len(obs), 6))
bigM[:, 0] = [o[1] for o in obs]
bigM[:, 1] = Z
bigM[:, 2] = bigM[:, 0] - bigM[:, 1]
bigM[:, 3] = [ 1 if o[0] == "rouge" else 0 for o in obs]
bigM[:, 4] = [ 1 if o[0] == "vert" else 0 for o in obs]
bigM[:, 5] = [ 1 if o[0] == "bleu" else 0 for o in obs]
bigM[:5]

array([[ 22.1606902 ,  20.54075598,   1.61993422,   0.        ,
          0.        ,   1.        ],
       [ 18.67214116,  20.54075598,  -1.86861482,   0.        ,
          0.        ,   1.        ],
       [  5.84770121,   6.30480401,  -0.4571028 ,   0.        ,
          1.        ,   0.        ],
       [ -0.21059849,   0.95499988,  -1.16559837,   1.        ,
          0.        ,   0.        ],
       [  1.51189831,   0.95499988,   0.55689843,   1.        ,
          0.        ,   0.        ]])

On utilise la fonction [cov](https://docs.scipy.org/doc/numpy/reference/generated/numpy.cov.html).

In [ ]:
c = numpy.cov(bigM.T)
c

array([[  8.97939782e+01,   8.87543481e+01,   1.03963003e+00,
         -3.95745034e+00,  -7.89879807e-01,   4.74733015e+00],
       [  8.87543481e+01,   8.87543481e+01,  -5.38408242e-16,
         -3.95745034e+00,  -7.89879807e-01,   4.74733015e+00],
       [  1.03963003e+00,  -5.38408242e-16,   1.03963003e+00,
          1.93569864e-17,  -2.65731819e-18,  -1.61084713e-17],
       [ -3.95745034e+00,  -3.95745034e+00,   1.93569864e-17,
          2.66666667e-01,  -8.88888889e-02,  -1.77777778e-01],
       [ -7.89879807e-01,  -7.89879807e-01,  -2.65731819e-18,
         -8.88888889e-02,   1.77777778e-01,  -8.88888889e-02],
       [  4.74733015e+00,   4.74733015e+00,  -1.61084713e-17,
         -1.77777778e-01,  -8.88888889e-02,   2.66666667e-01]])

On affiche un peu mieux les résultats :

In [ ]:
import pandas
pandas.DataFrame(c).applymap(lambda x: '%1.3f' % x)

,0,1,2,3,4,5
0,89.794,88.754,1.040,-3.957,-0.790,4.747
1,88.754,88.754,-0.000,-3.957,-0.790,4.747
2,1.040,-0.000,1.040,0.000,-0.000,-0.000
3,-3.957,-3.957,0.000,0.267,-0.089,-0.178
4,-0.790,-0.790,-0.000,-0.089,0.178,-0.089
5,4.747,4.747,-0.000,-0.178,-0.089,0.267


### Q10 

On permute rouge et vert. Construire le vecteur $W_i = \mathbb{E}(Y | rouge)\mathbb{1}_{X_i = vert} + \mathbb{E}(Y | vert)\mathbb{1}_{X_i = rouge} + \mathbb{E}(Y | bleu)\mathbb{1}_{X_i = bleu}$. Utiliser le résultat de la question 3 pour calculer les coefficients de la régression $Y_i = a^* W_i + b^*$. Vérifiez que l'erreur est supérieure.

In [ ]:
moys = moyenne_cat(obs)
try:
    moys["rouge"], moys["vert"] = moys["vert"], moys["rouge"]
except KeyError:
    # avec le hasard rien n'est jamais parfait
    print("keys:", list(moys.keys()))
    print(moys)
    raise KeyError('Arrêtons-nous...')
W = [moys[c] for c, y in obs]
obs3 = [(w, y) for (c, y), w in zip(obs, W)]
calcule_ab( obs3 )

(0.8093385511332646, 1.0138208507294095)

In [ ]:
def calcule_erreur(obs):
    a, b = calcule_ab(obs)
    e = [(a*x + b - y)**2 for x, y in obs]
    return sum(e) / len(obs)

calcule_erreur(obs2), calcule_erreur(obs3)

(0.9356670260144604, 19.044679603501358)

C'est carrément supérieur.

## Conclusion

L'[analyse des correspondances multiples](https://fr.wikipedia.org/wiki/Analyse_des_correspondances_multiples) est une façon d'étudier les modalités de variables catégorielles mais cela ne fait pas de la prédiction. Le modèle [logit - probit](https://fr.wikipedia.org/wiki/Mod%C3%A8le_Probit) prédit une variable binaire à partir de variables continue mais dans notre cas, c'est la variable à prédire qui est continue. Pour effectuer une prédiction, il convertit les catégories en variables numériques (voir [Categorical Variables](https://en.wikipedia.org/wiki/Categorical_variable)). Le langage R est plus outillé pour cela : [Regression on categorical variables](https://www.r-bloggers.com/regression-on-categorical-variables/). Le module [categorical-encoding](https://github.com/scikit-learn-contrib/categorical-encoding) est disponible en python. Cet examen décrit une méthode parmi d'autres pour transformer les catégories en variables continues.